In [1]:
%reset -f
import time
import pickle
import gym
from scipy.interpolate import UnivariateSpline
from stable_baselines3 import DDPG
import matplotlib.pyplot as plt
from stable_baselines3.common.env_checker import check_env
import numpy as np
from tqdm import tqdm
import derivatives as d
import forward_helpers as fh
from chameleon import Chameleon

In [6]:
n_elems = 10
steps = 1000
chameleon = Chameleon(E=1, n_elems=n_elems)

In [3]:
check_env(chameleon)

/Users/iandavenport/Desktop/learnrl/venv/lib/python3.7/site-packages/stable_baselines3/common/env_checker.py:273: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


[0.42846936 0.2765398  0.9082336 ]
[0.42846936 0.47040876 0.53477369 0.62156414 0.7307801  0.86242159
 1.0164886  1.19298113 1.39189918 1.61324275]


ValueError: x must be increasing if s > 0

In [4]:
breaking_action = [0.42846936, 0.2765398, 0.9082336]

In [ ]:
breaking

In [5]:
active_stress = np.array(
    [
        0.42846936,
        0.47040876,
        0.53477369,
        0.62156414,
        0.7307801,
        0.86242159,
        1.0164886,
        1.19298113,
        1.39189918,
        1.61324275,
    ]
)
# np.save("breaking_stress", active_stress)

In [7]:
fh.forward_simulate(chameleon, active_stress, sim_steps=steps)

ValueError: x must be increasing if s > 0

In [8]:
chameleon.position_history

deque([array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
              0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
       array([0.        , 0.11158948, 0.22290242, 0.33421536, 0.4455283 ,
              0.55684124, 0.66815418, 0.77946713, 0.89078007, 0.82264198])])

In [9]:
chameleon.displacement_history

deque([array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
       array([ 0.        ,  0.00047837,  0.0006802 ,  0.00088203,  0.00108386,
               0.00128569,  0.00148752,  0.00168935,  0.00189118, -0.17735802])])

In [ ]:
# # Create environment
# env = Chameleon(E=10, n_elems=n_elems)

# # Instantiate the agent
# model = DDPG("MlpPolicy", env, verbose=1)
# # Train the agent
# model.learn(total_timesteps=int(20))

## Is this is too slow? In order to do RL I may need to be able to simulate much faster than this. (Jax?)

## There seems to be an issue when the stress is not monotonically increasing (actually more complex than this). The second element often ends up behind the first and as a result the sim crashes. I am not exactly sure why it is happening but it is clearly related to the boundary condition being imposed on the first element. If I then try to make sure the second element is always beyond the first, the third element goes behind the second. I think the issue is related to the stress on the first element causing a high force since the 0->1 element gets stretched a lot and it may be less than the stress from the 1->2 element. If the active stress is not enough to overcome that, then the second element would move to the left and maybe end up behind the first element. 

Actually i don't buy that argument because then a uniform stress might also cause this issue. In that case we would have a large stress from the stretching 0->1 element and if the active stress wasn't enough to overcome it we could end up with the same issue. But whenever I run uniform simulations I don't see that problem (no matter the magnitude of the active stress).

One way to debug is to save the NET force (and component forces maybe) on each element at each point in time and to see what happens. 

# I think its the Derivative of $\sigma$ that's important here.

## Why do the elements come in after the first time step? 

Then after the first time step is over they move out as expected. Strange. It's only the element on the end and it comes from satisfying the boundary condition.

In [ ]:
chameleon.position_history[0]

In [ ]:
chameleon.position_history[1]

In [ ]:
chameleon.position_history[2]

# some hacky code to store the position history for later visualization.


In [ ]:
history = []
for i in range(len(chameleon.position_history)):
    x_p = chameleon.position_history[i]
    y = np.zeros(chameleon.n_elems)
    ar = np.vstack((x_p, y)).T
    history.append(ar)

with open("history.txt", "wb") as fp:
    pickle.dump(history, fp)